In [21]:
import yfinance as yf  # https://pypi.org/project/yfinance/

In [224]:
tickers = [
    "AIAG.L", # L&G Artificial Intelligence UCITS ETF (AIAG.L)
    "NBTK.DE",  # Invesco NASDAQ Biotech UCITS ETF (NBTK.DE)
    "XMLH.DE", # L&G Healthcare Breakthrough UCITS ETF (XMLH.DE)
    "ROBO", # Robo Global Robotics and Automation Index ETF (ROBO)
    "TIUP.DE", # Lyxor Core US TIPS (DR) UCITS ETF (TIUP.DE)
    "VWRL.AS", # Vanguard FTSE All-World UCITS ETF (VWRL.AS)
    "VFEM.AS", # Vanguard FTSE Emerging Markets UCITS ETF (VFEM.AS)
    "DXSA.DE", # Xtrackers Euro Stoxx Quality Dividend UCITS ETF (DXSA.DE)
    "SXR8.DE", # iShares VII PLC - iShares Core S&P 500 UCITS ETF (SXR8.DE)
    "IPRP.AS", # iShares European Property Yield UCITS ETF (IPRP.AS)
    "2B78.DE", # iShares Healthcare Innovation UCITS ETF (2B78.DE)
]
no_years = 5

In [225]:
dfs = []
# Create a list of DataFrames
for tick in tickers:
    df = yf.Ticker(tick).history(period=f"{no_years}y")
    df["Change"] = df["Close"].pct_change()*100
    df.drop(index=df.index[0], axis=0, inplace=True)  # no change on 1st day
    df.reset_index(inplace=True)  # make idx a col
    df = df[["Date", "Close", "Change"]]
    df["Tick"] = tick
    df["Date"] = df["Date"].dt.date
    dfs.append(df)
# tick.history_metadata

In [226]:
# Create dataframe with every day in last year
import pandas as pd
from datetime import datetime, timedelta

# get today's date
today = datetime.today()

# calculate the date one year ago
one_year_ago = today - timedelta(days=365*no_years)

# create a date range from one year ago to today
dates = pd.date_range(start=one_year_ago, end=today)

# create a DataFrame with the dates
new_df = pd.DataFrame(dates, columns=['Date'])
new_df = new_df[~new_df['Date'].dt.dayofweek.isin([5, 6])]

# you might want to remove the time part, if so, uncomment the next line
new_df['Date'] = new_df['Date'].dt.date
dates = new_df.copy()['Date'].values.tolist()

# Initialise
new_df["Tick"] = ""
new_df["Change"] = 0
new_df["Price"] = 0

In [227]:
for date in dates:
    # For each day in past calendar year
    most_min = 1000
    keep = False
    for temp_df in dfs:
        # For each ticker df
        if not temp_df[temp_df["Date"] == date].empty:
            minimum = temp_df[temp_df["Date"] == date]["Change"].item()
            if minimum < most_min:
                most_min = minimum
                keep = temp_df[temp_df["Date"] == date]

    if keep is not False:
        # print(keep["Close"].item())
        new_df.loc[new_df['Date'] == date, 'Price'] = keep["Close"].item()
        new_df.loc[new_df['Date'] == date, 'Change'] = keep["Change"].item()
        new_df.loc[new_df['Date'] == date, 'Tick'] = keep["Tick"].item()

new_df = new_df[new_df["Tick"] != ""]
new_df = new_df[new_df["Price"] != 0.0]


In [186]:
new_df.to_csv("research.csv")

In [228]:
totals = {key: 0 for key in tickers}
total_invested = 0
daily_invest = 10000 # 100 euros
for index, row in new_df.iterrows():
    total_invested += daily_invest
    buy_amt = daily_invest / row["Price"]
    totals[row["Tick"]] += buy_amt

In [229]:
totals

{'AIAG.L': 1309.1248633230198,
 'NBTK.DE': 29272.399265914097,
 'XMLH.DE': 64282.99486910078,
 'ROBO': 48520.49500395412,
 'TIUP.DE': 21745.72208987362,
 'VWRL.AS': 972.114753664455,
 'VFEM.AS': 23856.472137813882,
 'DXSA.DE': 48574.82862434,
 'SXR8.DE': 1331.7235408518288,
 'IPRP.AS': 56423.91414001854,
 '2B78.DE': 123962.54535624516}

In [230]:
total_value = 0
for k, v in totals.items():
    total_value += yf.Ticker(k).info["ask"] * v

In [231]:
total_value / 100 # In euros

105482.77554383867

In [232]:
total_invested / 100 # In euros

129200.0